In [ ]:
import pygame
import random

# Define some constants for the game
GRID_SIZE = 4
TILE_SIZE = 100
WINDOW_SIZE = TILE_SIZE * GRID_SIZE
BG_COLOR = (187, 173, 160)
TILE_COLORS = {
    0: (205, 193, 180),
    2: (238, 228, 218),
    4: (237, 224, 200),
    8: (242, 177, 121),
    16: (245, 149, 99),
    32: (246, 124, 95),
    64: (246, 94, 59),
    128: (237, 207, 114),
    256: (237, 204, 97),
    512: (237, 200, 80),
    1024: (237, 197, 63),
    2048: (237, 194, 46)
}

# utility functions

def get_direction_text(direction):
    return "up" if direction == 0 else "right" if direction == 1 else "down" if direction == 2 else "left" if direction == 3 else direction

In [ ]:
class Game:
    def __init__(self, gui=True, grid=None):
        # the gui flag is used to determine whether to render the game or not (might be useful for testing)
        # Initialize the game
        if gui:
            self.gui = True
            pygame.init()
            self.window = pygame.display.set_mode((WINDOW_SIZE, WINDOW_SIZE))
            self.font = pygame.font.SysFont("Arial", 32)
        if not gui:
            self.gui = False
        self.score = 0
        if grid:
            self.grid = grid
        else:
            self.reset()

    def reset(self):
        self.grid = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        self.add_random_tile()
        self.add_random_tile()
        # game over after 1 move for testing
        # self.grid = [
        #     [2, 4, 8, 16],
        #     [32, 64, 128, 256],
        #     [512, 1024, 2048, 256],
        #     [2, 4, 8, 16]
        # ]
        self.score = 0

    def add_random_tile(self):
        # Add a random tile to the grid, probability of adding a 2 is 90% and 4 is 10%
        empty_tiles = []
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                if self.grid[i][j] == 0:
                    empty_tiles.append((i, j))
        if empty_tiles:
            i, j = random.choice(empty_tiles)
            self.grid[i][j] = 2 if random.random() < 0.9 else 4

    def render(self):
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                # draw the tile
                pygame.draw.rect(self.window, TILE_COLORS[self.grid[i][j]], (j * TILE_SIZE, i * TILE_SIZE, TILE_SIZE, TILE_SIZE))

                # add the text to the tile
                text = self.font.render(str(self.grid[i][j]) if self.grid[i][j] else "", True, (119, 110, 101))
                text_rect = text.get_rect()
                text_rect.center = (j * TILE_SIZE + TILE_SIZE / 2, i * TILE_SIZE + TILE_SIZE / 2)
                self.window.blit(text, text_rect)

                # add a border around the tiles
                pygame.draw.rect(self.window, (187, 173, 160), (j * TILE_SIZE, i * TILE_SIZE, TILE_SIZE, TILE_SIZE), 5)

        pygame.display.update()

    def move(self, direction):
        # direction: 0 - up, 1 - right, 2 - down, 3 - left
        # slide the tiles in the given direction
        merged = [[False for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        initial_grid = [row[:] for row in self.grid]

        # Slide tiles as far as possible in the given direction, merging tiles of the same value once.
        if direction == 0:
            for i in range(GRID_SIZE):
                for j in range(GRID_SIZE):
                    shift = 0
                    for k in range(i): # k checks all tiles between i and 0
                        if self.grid[k][j] == 0:
                            shift += 1
                    
                    if shift:
                        self.grid[i - shift][j] = self.grid[i][j]
                        self.grid[i][j] = 0

                    if i - shift - 1 >= 0 and self.grid[i - shift - 1][j] == self.grid[i - shift][j] and not merged[i - shift - 1][j] and not merged[i - shift][j]:
                        self.score += self.grid[i - shift][j] * 2
                        self.grid[i - shift - 1][j] *= 2
                        self.grid[i - shift][j] = 0
                        merged[i - shift - 1][j] = True

        elif direction == 1:
            for i in range(GRID_SIZE):
                for j in range(GRID_SIZE - 1, -1, -1):
                    shift = 0
                    for k in range(j + 1, GRID_SIZE):
                        if self.grid[i][k] == 0:
                            shift += 1

                    if shift:
                        self.grid[i][j + shift] = self.grid[i][j]
                        self.grid[i][j] = 0

                    if j + shift + 1 < GRID_SIZE and self.grid[i][j + shift + 1] == self.grid[i][j + shift] and not merged[i][j + shift + 1] and not merged[i][j + shift]:
                        self.score += self.grid[i][j + shift] * 2
                        self.grid[i][j + shift + 1] *= 2
                        self.grid[i][j + shift] = 0
                        merged[i][j + shift + 1] = True

        elif direction == 2:
            for i in range(GRID_SIZE - 1, -1, -1):
                for j in range(GRID_SIZE):
                    shift = 0
                    for k in range(i + 1, GRID_SIZE):
                        if self.grid[k][j] == 0:
                            shift += 1

                    if shift:
                        self.grid[i + shift][j] = self.grid[i][j]
                        self.grid[i][j] = 0

                    if i + shift + 1 < GRID_SIZE and self.grid[i + shift + 1][j] == self.grid[i + shift][j] and not merged[i + shift + 1][j] and not merged[i + shift][j]:
                        self.score += self.grid[i + shift][j] * 2
                        self.grid[i + shift + 1][j] *= 2
                        self.grid[i + shift][j] = 0
                        merged[i + shift + 1][j] = True

        elif direction == 3:
            for i in range(GRID_SIZE):
                for j in range(GRID_SIZE):
                    shift = 0
                    for k in range(j):
                        if self.grid[i][k] == 0:
                            shift += 1

                    if shift:
                        self.grid[i][j - shift] = self.grid[i][j]
                        self.grid[i][j] = 0

                    if j - shift - 1 >= 0 and self.grid[i][j - shift - 1] == self.grid[i][j - shift] and not merged[i][j - shift - 1] and not merged[i][j - shift]:
                        self.score += self.grid[i][j - shift] * 2
                        self.grid[i][j - shift - 1] *= 2
                        self.grid[i][j - shift] = 0
                        merged[i][j - shift - 1] = True

        if self.grid != initial_grid:
            self.add_random_tile()
            # print("Game State after moving " + get_direction_text(direction) + ":")
            # print(str(self))
            # print("Score: " + str(self.score))
            # print()
            if self.check_game_over():
                if self.gui:
                    pygame.time.delay(5000)
                    pygame.quit()
                return (1, self.grid)
            if self.gui:
                self.render()
            return (0, self.grid)
        else:
            return (2, self.grid)

    def check_game_over(self):
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                if self.grid[i][j] == 0:
                    return
                if i > 0 and self.grid[i - 1][j] == self.grid[i][j]:
                    return
                if i < GRID_SIZE - 1 and self.grid[i + 1][j] == self.grid[i][j]:
                    return
                if j > 0 and self.grid[i][j - 1] == self.grid[i][j]:
                    return
                if j < GRID_SIZE - 1 and self.grid[i][j + 1] == self.grid[i][j]:
                    return

        # overlay text on the screen
        if self.gui:
            text = self.font.render("Game Over!", True, (119, 110, 101))
            text_rect = text.get_rect()
            text_rect.center = (WINDOW_SIZE / 2, WINDOW_SIZE / 2)
            self.window.blit(text, text_rect)
            pygame.display.update()
        return True
        

    def __str__(self):
        return "\n".join([" ".join([str(self.grid[i][j]) for j in range(GRID_SIZE)]) for i in range(GRID_SIZE)])

### Run the cell below to play the game manually

In [ ]:
game = Game()
game.render()

# close the game window when the user presses the close button
done = False
while not done:
    # game loop
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.display.quit()
            pygame.quit()
            print("Exiting...")
            done = True
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:
                game.reset()
                game.render()
            elif event.key == pygame.K_UP:
                if game.move(0)[0] == 1:
                    done = True
            elif event.key == pygame.K_RIGHT:
                if game.move(1)[0] == 1:
                    done = True
            elif event.key == pygame.K_DOWN:
                if game.move(2)[0] == 1:
                    done = True
            elif event.key == pygame.K_LEFT:
                if game.move(3)[0] == 1:
                    done = True
        else:
            continue

### Random moves simulation

In [69]:
game = Game()
game.render()

done = False
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.display.quit()
            pygame.quit()
            print("Exiting...")
            done = True
    pygame.time.delay(200)
    if game.move(random.randint(0, 3))[0] == 1:
        done = True
        

In [70]:
# stats for 1000 random games
Sum = 0
Max = 0
Max_game = None
_2prob = 0
_4prob = 0
_8prob = 0
_16prob = 0
_32prob = 0
_64prob = 0
_128prob = 0
_256prob = 0
_512prob = 0
_1024prob = 0
_2048prob = 0
_4096prob = 0
for i in range(1000):
    game = Game(gui=False)
    while not game.check_game_over():
        game.move(random.randint(0, 3))
    Sum += game.score
    Max = max(Max, game.score)
    if Max == game.score:
        Max_game = game
    if any(2048 in row for row in game.grid):
        _2048prob += 1
    if any(1024 in row for row in game.grid):
        _1024prob += 1
    if any(512 in row for row in game.grid):
        _512prob += 1
    if any(256 in row for row in game.grid):
        _256prob += 1
    if any(128 in row for row in game.grid):
        _128prob += 1
    if any(64 in row for row in game.grid):
        _64prob += 1
    if any(32 in row for row in game.grid):
        _32prob += 1
    if any(16 in row for row in game.grid):
        _16prob += 1
    if any(8 in row for row in game.grid):
        _8prob += 1
    if any(4 in row for row in game.grid):
        _4prob += 1
    if any(2 in row for row in game.grid):
        _2prob += 1

print("Average score: " + str(Sum / 1000))
print("Maximum score: " + str(Max))
print("Maximum score grid: ")
print(str(Max_game))
print("2 probability: " + str(_2prob / 1000))
print("4 probability: " + str(_4prob / 1000))
print("8 probability: " + str(_8prob / 1000))
print("16 probability: " + str(_16prob / 1000))
print("32 probability: " + str(_32prob / 1000))
print("64 probability: " + str(_64prob / 1000))
print("128 probability: " + str(_128prob / 1000))
print("256 probability: " + str(_256prob / 1000))
print("512 probability: " + str(_512prob / 1000))
print("1024 probability: " + str(_1024prob / 1000))
print("2048 probability: " + str(_2048prob / 1000))
print("4096 probability: " + str(_4096prob / 1000))

Average score: 1087.408
Maximum score: 3084
Maximum score grid: 
4 8 2 16
64 256 32 2
4 8 128 8
2 16 4 2
2 probability: 1.0
4 probability: 1.0
8 probability: 0.987
16 probability: 0.948
32 probability: 0.855
64 probability: 0.704
128 probability: 0.482
256 probability: 0.085
512 probability: 0.0
1024 probability: 0.0
2048 probability: 0.0
4096 probability: 0.0


## Uninformed search on the state space

### BFS

In [71]:
game = Game(gui=False)

def bfs(game):
    # breadth first search to find the best move
    # returns a tuple (best_score, moves)
    # moves: list of moves taken to reach the best score
    # game: the game object
    queue = []
    visited = set()
    queue.append((game.grid, 0))
    visited.add(str(game.grid))
    best_score = 0
    moves = [] # list of moves taken to reach the best score
    while queue:
        grid, score = queue.pop(0)
        if score > best_score:
            best_score = score
        for i in range(4):
            game.grid = grid
            game.move(i)
            if str(game.grid) not in visited:
                queue.append((game.grid, score + game.score))
                moves.append(i)
                visited.add(str(game.grid))
    return (best_score, moves)

print("Game State:\n", str(game))
(game.score, moves) = bfs(game)
print("Moves: " + str([get_direction_text(move) for move in moves]))
print("Final State:\n", str(game))
print("Score: " + str(game.score))
print("Max Tile: " + str(max(max(row) for row in game.grid)))    

Game State:
 0 0 2 0
0 0 0 2
0 0 0 0
0 0 0 0
Moves: ['up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'd

In [73]:
# stats for 1000 random games
Sum = 0
Max = 0
Max_game = None
_2prob = 0
_4prob = 0
_8prob = 0
_16prob = 0
_32prob = 0
_64prob = 0
_128prob = 0
_256prob = 0
_512prob = 0
_1024prob = 0
_2048prob = 0
_4096prob = 0

for i in range(1000):
    game = Game(gui=False)
    (game.score, moves) = bfs(game)
    Sum += game.score
    Max = max(Max, game.score)
    if Max == game.score:
        Max_game = game
    if any(2048 in row for row in game.grid):
        _2048prob += 1
    if any(1024 in row for row in game.grid):
        _1024prob += 1
    if any(512 in row for row in game.grid):
        _512prob += 1
    if any(256 in row for row in game.grid):
        _256prob += 1
    if any(128 in row for row in game.grid):
        _128prob += 1
    if any(64 in row for row in game.grid):
        _64prob += 1
    if any(32 in row for row in game.grid):
        _32prob += 1
    if any(16 in row for row in game.grid):
        _16prob += 1
    if any(8 in row for row in game.grid):
        _8prob += 1
    if any(4 in row for row in game.grid):
        _4prob += 1
    if any(2 in row for row in game.grid):
        _2prob += 1

print("Average score: " + str(Sum / 1000))
print("Maximum score: " + str(Max))
print("Maximum score grid: ")
print(str(Max_game))
print("2 probability: " + str(_2prob / 1000))
print("4 probability: " + str(_4prob / 1000))
print("8 probability: " + str(_8prob / 1000))
print("16 probability: " + str(_16prob / 1000))
print("32 probability: " + str(_32prob / 1000))
print("64 probability: " + str(_64prob / 1000))
print("128 probability: " + str(_128prob / 1000))
print("256 probability: " + str(_256prob / 1000))
print("512 probability: " + str(_512prob / 1000))
print("1024 probability: " + str(_1024prob / 1000))
print("2048 probability: " + str(_2048prob / 1000))
print("4096 probability: " + str(_4096prob / 1000))


Average score: 2687.344
Maximum score: 7728
Maximum score grid: 
4 64 8 2
32 8 256 16
16 512 2 32
4 8 4 2
2 probability: 1.0
4 probability: 0.998
8 probability: 0.99
16 probability: 0.962
32 probability: 0.891
64 probability: 0.8
128 probability: 0.713
256 probability: 0.501
512 probability: 0.053
1024 probability: 0.0
2048 probability: 0.0
4096 probability: 0.0


### DFS

In [ ]:
game = Game(gui=False)

def dfs(game):
    

print("Game State:\n", str(game))
(game.score, moves) = dfs(game)
print("Moves: " + str([get_direction_text(move) for move in moves]))
print("Final State:\n", str(game))
print("Score: " + str(game.score))
print("Max Tile: " + str(max(max(row) for row in game.grid)))




### Informed search on the state space

Before performing informed search, we need to define a heuristic function.
Some helpful functions are provided below.


In [12]:
def largest_tile(grid):
    # returns the largest tile in the grid
    return max(max(row) for row in grid)

def empty_tiles(grid):
    # returns the number of empty tiles in the grid, more empty tiles provide more flexibility
    return sum(1 for row in grid for tile in row if tile == 0)

def largest_tile_in_corner(grid):
    largest_tile = max(max(row) for row in grid)

    if grid[0][0] == largest_tile or grid[0][GRID_SIZE - 1] == largest_tile or grid[GRID_SIZE - 1][0] == largest_tile or grid[GRID_SIZE - 1][GRID_SIZE - 1] == largest_tile:
        return 1
    else:
        return 0

def final_heuristic(grid):
    return largest_tile(grid) + empty_tiles(grid) + largest_tile_in_corner(grid)


#### A* search with different heuristic functions

In [13]:
import heapq
import copy

class GameNode:
    def __init__(self, game, path, cost, heuristic):
        self.game = game
        self.path = path
        self.cost = cost
        self.heuristic = heuristic

    def __lt__(self, other):
        return (self.cost + self.heuristic) > (other.cost + other.heuristic)
    
def a_star(initial_game):
    initial_node = GameNode(initial_game, ["initial"], 0, final_heuristic(initial_game.grid))
    open_set = [initial_node]
    closed_set = set()

    explored_states = []

    while open_set:
        # print("=====================================")
        # for node in open_set:
        #     print(node.game, node.heuristic, [get_direction_text(move) for move in node.path])
        # print("=====================================\n\n")
        current_node = heapq.heappop(open_set)

        if current_node.game.check_game_over():
            return (current_node.game.score, current_node.path, current_node.game.grid, explored_states)
        
        closed_set.add(str(current_node.game.grid))
        explored_states.append(current_node.game.grid)

        for i in range(4):
            new_game = copy.deepcopy(current_node.game)
            result, grid = new_game.move(i)

            if result == 2:
                continue # skip this move if it doesn't change the grid

            if str(grid) not in closed_set:
                new_path = current_node.path[:] + [i]
                new_cost = current_node.cost + 1
                new_heuristic = final_heuristic(new_game.grid)
                new_node = GameNode(new_game, new_path, new_cost, new_heuristic)
                heapq.heappush(open_set, new_node)

    return (0, [])
    

game = Game(gui=False)
print("Game State:\n"+ str(game))
(score, moves, final_state, explored_states) = a_star(game)
moves = [get_direction_text(move) for move in moves]
print("Moves: " + str(moves))
print("\nFinal State:")
for row in final_state:
    print(row)
print("\nScore: " + str(score))

print("Explored States:")
for i, state in enumerate(explored_states):
    print("Heuristic: " + str(final_heuristic(state)))
    for row in state:
        print(row)
    print()

Game State:
0 0 0 0
0 0 0 0
0 0 2 0
2 0 0 0
Moves: ['initial', 'up', 'right', 'up', 'up', 'right', 'up', 'right', 'right', 'right', 'up', 'right', 'up', 'right', 'right', 'right', 'up', 'up', 'right', 'up', 'down', 'down', 'right', 'down', 'up', 'right', 'up', 'right', 'up', 'up', 'right', 'down', 'right', 'left', 'right', 'up', 'up', 'up', 'right', 'up', 'right', 'right', 'up', 'up', 'up', 'right', 'up', 'up', 'up', 'up', 'up', 'right', 'right', 'right', 'up', 'right', 'right', 'up', 'up', 'up', 'left', 'up', 'up', 'right', 'up', 'right', 'right', 'up', 'right', 'down', 'right', 'up', 'up', 'up', 'right', 'up', 'up', 'right', 'up', 'up', 'down', 'right', 'up', 'right', 'up', 'up', 'right', 'up', 'right', 'up', 'right', 'down', 'right', 'up', 'right', 'right', 'up', 'up', 'left', 'down', 'right', 'down', 'right', 'down', 'down', 'right', 'down', 'right', 'right', 'down', 'down', 'right', 'left', 'right', 'down', 'down', 'right', 'right', 'down', 'right', 'down', 'right', 'right', 'left

In [14]:
# a star performance stats
import time

Sum = 0
Max = 0
Max_game = None
_2prob = 0
_4prob = 0
_8prob = 0
_16prob = 0
_32prob = 0
_64prob = 0
_128prob = 0
_256prob = 0
_512prob = 0
_1024prob = 0
_2048prob = 0
_4096prob = 0

init_time = time.time()
for i in range(1000):
    game = Game(gui=False)
    (score, moves, final_state, explored_states) = a_star(game)
    Sum += score
    Max = max(Max, score)
    if Max == score:
        Max_game = final_state
    if any(2048 in row for row in final_state):
        _2048prob += 1
    if any(1024 in row for row in final_state):
        _1024prob += 1
    if any(512 in row for row in final_state):
        _512prob += 1
    if any(256 in row for row in final_state):
        _256prob += 1
    if any(128 in row for row in final_state):
        _128prob += 1
    if any(64 in row for row in final_state):
        _64prob += 1
    if any(32 in row for row in final_state):
        _32prob += 1
    if any(16 in row for row in final_state):
        _16prob += 1
    if any(8 in row for row in final_state):
        _8prob += 1
    if any(4 in row for row in final_state):
        _4prob += 1
    if any(2 in row for row in final_state):
        _2prob += 1
final_time = time.time()

print("Average score: " + str(Sum / 1000))
print("Maximum score: " + str(Max))
print("Maximum score grid: \n")
for row in Max_game:
    print(row)
print()
print("Average time per game (seconds): " + str((final_time - init_time) / 1000))
print("2 probability: " + str(_2prob / 1000))
print("4 probability: " + str(_4prob / 1000))
print("8 probability: " + str(_8prob / 1000))
print("16 probability: " + str(_16prob / 1000))
print("32 probability: " + str(_32prob / 1000))
print("64 probability: " + str(_64prob / 1000))
print("128 probability: " + str(_128prob / 1000))
print("256 probability: " + str(_256prob / 1000))
print("512 probability: " + str(_512prob / 1000))
print("1024 probability: " + str(_1024prob / 1000))
print("2048 probability: " + str(_2048prob / 1000))
print("4096 probability: " + str(_4096prob / 1000))

Average score: 4812.672
Maximum score: 14744
Maximum score grid: 

[1024, 512, 2, 64]
[16, 32, 64, 32]
[8, 16, 128, 16]
[2, 4, 8, 4]

Average time per game (seconds): 0.07292650485038757
2 probability: 1.0
4 probability: 0.998
8 probability: 0.992
16 probability: 0.97
32 probability: 0.934
64 probability: 0.882
128 probability: 0.792
256 probability: 0.752
512 probability: 0.332
1024 probability: 0.02
2048 probability: 0.0
4096 probability: 0.0
